### Digit Classification Using Dirichlet-Multinomial Models (MM and EM methods)

### Wenxi Yu

### 1. Introduction

* Goals: This homework aims to build a classifier for handwritten digits using Dirichlet-multinomial models. MM and EM methods are used for maximum likelihood estimation (MLE).

* Programming language: Julia

* Setup: Each digit is represented by a  32×32  bitmap in which each element indicates one pixel with a value of white or black. Each  32×32  bitmap is divided into blocks of  4×4 , and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector  x=(x1,…,x64)  of length 64 where each element is a count between 0 and 16.

### 2. Julia Codes

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using MM algorithm.

In [1]:
# download file if it's not in current folder
if !isfile("optdigits.tra")
    download("http://hua-zhou.github.io/teaching/" * 
        "biostatm280-2017spring/hw/optdigits.tra")
end
if !isfile("optdigits.tes")
    download("http://hua-zhou.github.io/teaching/" * 
        "biostatm280-2017spring/hw/optdigits.tes")
end

traindata = readcsv("optdigits.tra", Int)
digit = traindata[:, end]
count = traindata[:, 1:64]' # make sure each column is a data point

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  550k  100  550k    0     0  1760k      0 --:--:-- --:--:-- --:--:-- 1804k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  258k  100  258k    0     0   703k      0 --:--:-- --:--:-- --:--:--  708k


LoadError: SystemError: opening file optdigits.tra: No such file or directory

In [2]:
# read the data
traindata = readcsv("/Users/Wenxi/Desktop/280/hw4/optdigits.tra", Int)
digit = traindata[:, end]
count = traindata[:, 1:64]'

64×3823 Array{Int64,2}:
  0   0   0   0   0   0   0   0   0  …   0   0   0   0   0   0   0   0   0
  1   0   0   0   0   0   0   0   0      0   0   1   0   0   0   0   0   0
  6  10   8   0   5  11   1   8  15      9   9  10   6   5   0   3   6   2
 15  16  15   3  14  16  11  10   2     16  16  16  16  13   1  15  16  15
 12   6  16  11   4  10  13   8  14      6  12  16  11  11  12   0   2  16
  1   0  13  16   0   1  11   7  13  …   0   1   4   0   2   1   0   0  13
  0   0   0   0   0   0   7   2   2      0   0   0   0   0   0   0   0   1
  0   0   0   0   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  7   7   1   0   0   4   0   1   0      2   3   8   1   2   0   0   0   0
 16  16  11   5  13  16   9  15  16  …  14  16  13  16  15   0  11  15   3
  6   8   9  16   8  10  14  14  15     16  10  12   2   6  14  14  10   7
  6  16  11  11   0  15   6  12  12     16  15  16  12   5  10   0   0  10
 

In [1]:
"""
    dirmult_mom(X)

Find the method of moment estimator of Dirichlet-multinomial distribution.
Each column of `X` is one data point.
"""
function dirmult_mom{T <: Real}(X::Matrix{T})
    d, n   = size(X)
    α      = zeros(Float64, d)
    pi1    = zeros(Float64, d)
    pi2    = zeros(Float64, d)
    total  = zero(T)
    for j in 1:n
        colsum = zero(T)
        for i in 1:d
            α[i]   += X[i, j] # accumulate row sum
            colsum += X[i, j]
        end
        total += colsum
        if colsum > 0
            for i in 1:d
                pi      = X[i, j] / colsum
                pi1[i] += pi   # accumulate pi
                pi2[i] += pi^2 # accumulate pi^2
            end  
        end
    end
    # estiamte ρ and p
    ρ = zero(Float64)
    for i in 1:d
        if pi1[i] > 0
            ρ += pi2[i] / pi1[i]
        end
        α[i] /= total
    end
    αsum = max((d - ρ) / (ρ - 1), 1e-6)
    for i in 1:d
        α[i] *= αsum
    end
    return α
end

dirmult_mom

In [4]:
"""
    dirmult_logpdf(x, α)

Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::AbstractVector, α::Vector)
    T = promote_type(eltype(x), eltype(α))
    xs = sum(x)
    αs = sum(α)
    if xs == 0 && αs == 0
        return zero(T)
    elseif xs > 0 && αs == 0
        return convert(T, -Inf)
    else
        l = lfact(xs) - lgamma(xs + αs) + lgamma(αs)
    end
    for i in eachindex(x)
        if α[i] == 0 && x[i] > 0
            return convert(T, -Inf)
        elseif α[i] > 0
            l += - lfact(x[i]) + lgamma(x[i] + α[i]) - lgamma(α[i])
        end
    end
    return l
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at a sample `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::AbstractMatrix, α::Vector)
    l  = zero(promote_type(eltype(X), eltype(α)))
    for j in 1:size(X, 2)
        l += dirmult_logpdf(view(X, :, j), α)
    end
    return l
end

dirmult_logpdf

In [5]:
"""
    dirmult_score(X, α)

Evaluate the score (gradient) of Dirichlet-multinomial log-likelihood at `α`.
Each column of `X` is one data point.
"""
function dirmult_score(X::AbstractMatrix, α::Vector)
    T = promote_type(eltype(X), eltype(α))
    ∇ = zeros(T, length(α))
    return dirmult_score!(∇, X, α)
end

function dirmult_score!(∇::Vector, X::AbstractMatrix, α::Vector)
    fill!(∇, zero(eltype(∇)))
    for j in 1:size(X, 2)
        dirmult_score!(∇, view(X, :, j), α)
    end
    return ∇
end

function dirmult_score!(
        ∇::Vector, 
        x::AbstractVector, 
        α::Vector
    )
    
    T = promote_type(eltype(x), eltype(α))
    xs = zero(eltype(x))
    αs = zero(eltype(α))
    for i in eachindex(x)
        xs += x[i]
        αs += α[i]
        if α[i] == 0 && x[i] > 0
            ∇[i] += Inf
        elseif α[i] > 0
            ∇[i] += digamma(x[i] + α[i]) - digamma(α[i])
        end
    end
    if αs > 0
        c = digamma(xs + αs) - digamma(αs)
        for i in eachindex(x)
            ∇[i] -= c
        end
    end
    return ∇
end

dirmult_score! (generic function with 2 methods)

In [6]:
"""
    dirmult_obsinfo(X, α)

Evaluate the observed information matrix of Dirichlet-multinomial log-likelihood
at `α`. Each column of `X` is one data point. Return vector `d` and constant `c`.
The observed information matrix equals `Diagonal(d) - c`.
"""
function dirmult_obsinfo(X::AbstractMatrix, α::Vector)
    T = promote_type(eltype(X), eltype(α))
    d = zeros(T, length(α))
    return dirmult_obsinfo!(d, X, α)
end

function dirmult_obsinfo!(d::Vector, X::AbstractMatrix, α::Vector)
    T = promote_type(eltype(X), eltype(α))
    c = zero(T)
    fill!(d, zero(eltype(d)))
    for j in 1:size(X, 2)
        c += dirmult_obsinfo!(d, view(X, :, j), α)
    end
    return c, d
end

function dirmult_obsinfo!(d::Vector, x::AbstractVector, α::Vector)
    T  = promote_type(eltype(x), eltype(α))
    xs = zero(eltype(x))
    αs = zero(eltype(α))
    for i in eachindex(x)
        xs += x[i]
        αs += α[i]
        if α[i] == 0 && x[i] > 0
            d[i] += T(Inf)
        elseif α[i] > 0
            d[i] += T(trigamma(α[i]) - trigamma(x[i] + α[i]))
        end
    end
    if αs == 0 && xs > 0
        c = T(Inf)
    elseif αs == 0 && xs == 0
        c = zero(T)
    elseif αs > 0
        c = T(trigamma(αs) - trigamma(xs + αs))
    end
    return c
end

dirmult_obsinfo! (generic function with 2 methods)

In [9]:
"""
    dirmult_mm(X)

Find the MLE of Dirichlet-multinomial distribution using MM algorithm.

# Argument
* `X`: an `d`-by-`n` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: starting point. 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
# Output
* `logl`: the log-likelihood at MLE.   
* `niter`: the number of iterations performed.
# `α`: the MLE.
* `∇`: the gradient at MLE. 
* `obsinfo`: the observed information matrix at MLE. 
"""
function dirmult_mm(
        X::AbstractMatrix; 
        α0::Vector = dirmult_mom(X), 
        maxiters::Int = 100, 
        tolfun = 1e-6,
        verbose::Bool = true
    )
    # first transpose X
    X = X'
    
    # only consider rows and columns with sum >0
    rowind = find(sum(X, 2))
    colind = find(sum(X, 1)) 
    Xwork = @view X[rowind, colind]
    αwork  = α0[colind]

    # preparation
    αnew = similar(αwork)
    n, d = size(Xwork)
    niter = zero(Int)
    loglold = dirmult_logpdf(Xwork', αwork)
    rowsums = sum(Xwork, 2)
    logliter = zero(Float64)
    colmax = maximum(Xwork, 1)

    for iter in 1:maxiters
        # first calculate the denominator
        denom = 0
        for k in 0:(maximum(rowsums) - 1)
            r = sum(rowsums .> k)
            denom = denom + r / (sum(αwork) + k)
        end
        
        # then calculate the numerator
        for j in 1:d
            tempt = 0
            if(colmax[j] >= 1)
                for k in 0:(colmax[j] - 1)
                    s = sum(Xwork[:, j] .> k)
                    tempt = tempt + s / (αwork[j] + k)
                end
                αnew[j] = tempt / denom * αwork[j]
            end
        end
        @show αnew
        @show size(αnew)
        logliter = dirmult_logpdf(Xwork', αnew)

        # print iterate log-L if requested
        if verbose
            println("iteration ", iter, ", logl = ", logliter)
        end
        
        copy!(αwork, αnew)
        if abs(logliter - loglold) < tolfun * (abs(loglold) + 1)
            niter = iter
            break
        end
        loglold = logliter
        niter = maxiters
    end

    # compute logl, gradient, Hessian from final iterate
    αfinal = zeros(eltype(α0), length(α0))
    αfinal[colind] = αwork
    
    ∇final = zeros(eltype(α0), length(α0))
    ∇final[colind] = dirmult_score(Xwork', αwork)

    obsinfo = zeros(eltype(α0), length(α0), length(α0))
    obsinfo_dinv = similar(αwork)
    obsinfo_c, obsinfo_d = dirmult_obsinfo!(obsinfo_dinv, Xwork', αwork)
    obsinfo[colind, colind] = diagm(obsinfo_d) - obsinfo_c
     
    return logliter, niter, αfinal, ∇final, obsinfo
end

dirmult_mm

In [10]:
dirmult_mm(count; verbose = false, maxiters = 1)

αnew = [0.0983901,1.2734,2.94045,2.83522,1.16647,0.214591,0.0190132,0.000476166,0.413715,2.53188,2.95498,2.62303,1.87409,0.379492,0.0236629,0.0014291,0.557417,2.21671,1.47302,1.55838,1.74181,0.385576,0.0107661,0.000634563,0.514227,2.09923,2.11071,2.18359,1.75435,0.443831,0.00158664,0.000793203,0.40474,1.65621,2.09181,2.40776,2.12019,0.62641,0.00623083,0.268091,1.27501,1.48604,1.72447,1.98605,0.707496,0.00763019,0.00508976,0.177413,1.79041,2.41015,2.36149,2.12311,0.723356,0.0335669,0.000158641,0.0801053,1.3356,2.96557,2.78469,1.45695,0.353534,0.0347627]
size(αnew) = (62,)


(-475812.5993711099,1,[0.0,0.0983901,1.2734,2.94045,2.83522,1.16647,0.214591,0.0190132,0.000476166,0.413715  …  0.723356,0.0335669,0.000158641,0.0801053,1.3356,2.96557,2.78469,1.45695,0.353534,0.0347627],[0.0,-46.4583,-205.976,-15.8135,-33.1534,-441.578,-779.161,-274.447,-141.011,-352.484  …  -643.416,-152.014,-140.864,-83.9284,-254.603,-23.9744,-86.0565,-390.857,-794.882,-235.846],
[0.0 0.0 … 0.0 0.0; 0.0 61500.7 … -43.6346 -43.6346; … ; 0.0 -43.6346 … 11254.9 -43.6346; 0.0 -43.6346 … -43.6346 1.71423e5])

In [2]:
using DataFrames, Distributions

ndigit       = zeros(Int, 10)
logl_dirmult = zeros(10)
iter_dirmult = zeros(Int, 10)
time_dirmult = zeros(10)
αhat_dirmult = zeros(64, 10)
logl_multnom = zeros(10)


for d in 0:9
    # retrieve data for digit d
    Xd = traindata[traindata[:, end] .== d, 1:64]'
    ndigit[d + 1] = size(Xd, 2)
    # fit Dirichlet-multinomial
    tic()
    logl_dirmult[d + 1], iter_dirmult[d + 1], αhat, = dirmult_mm(Xd; verbose = false)
    αhat_dirmult[:, d + 1] = αhat
    time_dirmult[d + 1] = toc()
end

LoadError: UndefVarError: traindata not defined

In [9]:
result = DataFrame(digit = 0:9, n = ndigit, 
    logl_dm = logl_dirmult, 
    iters = iter_dirmult, runtime = time_dirmult)
print(result)

10×5 DataFrames.DataFrame
│ Row │ digit │ n   │ logl_dm  │ iters │ runtime   │
├─────┼───────┼─────┼──────────┼───────┼───────────┤
│ 1   │ 0     │ 376 │ -37361.9 │ 100   │ 0.794803  │
│ 2   │ 1     │ 389 │ -42179.5 │ 47    │ 0.383425  │
│ 3   │ 2     │ 380 │ -39985.3 │ 6     │ 0.0466663 │
│ 4   │ 3     │ 389 │ -40519.9 │ 48    │ 0.434473  │
│ 5   │ 4     │ 387 │ -43489.0 │ 26    │ 0.202164  │
│ 6   │ 5     │ 376 │ -41191.6 │ 36    │ 0.282102  │
│ 7   │ 6     │ 377 │ -37703.0 │ 16    │ 0.117468  │
│ 8   │ 7     │ 387 │ -40304.1 │ 6     │ 0.045039  │
│ 9   │ 8     │ 380 │ -43131.3 │ 20    │ 0.187638  │
│ 10  │ 9     │ 382 │ -43709.9 │ 39    │ 0.333364  │

In [11]:
using MLBase

# read in test digits
testdata = readcsv("optdigits.tes", Int)
Xtest    = testdata[:, 1:64]'
ytest    = testdata[:, 65]

# matrix of assignment probability
test_digit_prob = [dirmult_logpdf(Xtest[:, j], αhat_dirmult[:, d]) 
    for d in 1:10, j in 1:size(Xtest, 2)]

# factor in prior probabilities
test_digit_prob .= test_digit_prob .+ log(ndigit / sum(ndigit))

# prediction
pred = [indmax(test_digit_prob[:, j]) for j in 1:size(Xtest, 2)]

# confusion matrix
confmat = confusmat(10, ytest + 1, pred)

10×10 Array{Int64,2}:
 174    0    0    0    4    0    0    0    0    0
   0  133   21    0    2    0    2    0   13   11
   0    9  151    2    0    1    1    1    8    4
   0    2    1  155    0    4    0    7    5    9
   0    2    0    0  173    0    1    2    2    1
   0    0    0    0    1  166    1    0    0   14
   0    6    0    0    2    1  170    0    2    0
   0    0    0    0   11    0    0  164    2    2
   0   22    1    0    1    1    1    1  134   13
   0    5    0    4    6    2    0    3    4  156

Compare the efficacy of Newton's method and MM algorithm:
- for one iteration, both of the methods take similar amount of time;
- however for MM algorithm, it normally takes more iterations to converge to the MLE;
- so Newton's method is much more efficient than MM algorithm in this problem.